In [1]:
import cv2
import os
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
from pygame import mixer
import urllib.request
from http.client import IncompleteRead

pygame 2.5.2 (SDL 2.28.3, Python 3.9.18)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:

# Initialize pygame mixer for sound alerts
mixer.init()
sound = mixer.Sound('car-lock.wav')

# Load face and eye cascades
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye.xml")

# Load eye detection model
model = load_model(r'C:\Users\LAPTOP\Desktop\CameraWebServer\python code\cnnCat2.h5')
lbl = ['Close', 'Open']

# URL of the ESP32 cam stream
url = 'http://192.168.43.109/cam-lo.jpg'
cv2.namedWindow("live Cam Testing", cv2.WINDOW_AUTOSIZE)

# Create a VideoCapture object
cap = cv2.VideoCapture(url)

# Check if the IP camera stream is opened successfully
if not cap.isOpened():
    print("Failed to open the IP camera stream")
    exit()

font = cv2.FONT_HERSHEY_COMPLEX_SMALL
score = 0

while True:
    try:
        # Read a frame from the IP camera stream
        img_resp = urllib.request.urlopen(url)
        imgnp = np.array(bytearray(img_resp.read()), dtype=np.uint8)
        im = cv2.imdecode(imgnp, -1)

        # Convert the frame to grayscale
        gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

        # Detect faces in the frame
        faces = face_cascade.detectMultiScale(gray, minNeighbors=3, scaleFactor=1.1, minSize=(25, 25))
        eyes = eye_cascade.detectMultiScale(gray, minNeighbors=1, scaleFactor=1.1)

        cv2.rectangle(im, (0, im.shape[0] - 50), (200, im.shape[0]), (0, 0, 0), thickness=cv2.FILLED)

        for (x, y, w, h) in eyes:
            eye = im[y:y + h, x:x + w]

            # Convert the eye to grayscale
            eye_gray = cv2.cvtColor(eye, cv2.COLOR_BGR2GRAY)

            # Resize the eye to (24, 24)
            eye_gray = cv2.resize(eye_gray, (24, 24))

            # Normalize the pixel values to be between 0 and 1
            eye_gray = eye_gray / 255.0

            # Reshape the eye to match the expected input shape of the model
            eye_gray = np.reshape(eye_gray, (1, 24, 24, 1))

            # Make a prediction using the model
            prediction = model.predict(eye_gray)

            # Rest of your code...
            # Condition for Close
            if prediction[0][0] > 0.30:
                cv2.putText(im, "Closed", (10, im.shape[0] - 20), font, 1, (255, 255, 255), 1, cv2.LINE_AA)
                cv2.putText(im, 'Score:' + str(score), (100, im.shape[0] - 20), font, 1, (255, 255, 255), 1, cv2.LINE_AA)
                score = score + 1
                if score > 20:
                    try:
                        sound.play()
                    except:
                        pass

            # Condition for Open
            elif prediction[0][1] > 0.70:
                score = score - 1
                if score < 0:
                    score = 0
                cv2.putText(im, "Open", (10, im.shape[0] - 20), font, 1, (255, 255, 255), 1, cv2.LINE_AA)
                cv2.putText(im, 'Score:' + str(score), (100, im.shape[0] - 20), font, 1, (255, 255, 255), 1, cv2.LINE_AA)

        # Display the processed frame
        cv2.imshow('live Cam Testing', im)

        # Check for the 'q' key to exit
        key = cv2.waitKey(5)
        if key == ord('q'):
            break

    except IncompleteRead:
        # Handle IncompleteRead error by continuing with the next iteration
        pass

# Release the camera and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 106ms/step
